In [0]:
import matplotlib.pyplot as plt
import numpy as np
import copy
import math
import csv
from google.colab import files
from collections import deque

In [44]:
uploaded = files.upload()

Saving mnist.csv to mnist (1).csv


In [45]:
file_data   = "mnist.csv"
handle_file = open(file_data, "r")
data        = handle_file.readlines()
handle_file.close()

size_row    = 28    # height of the image
size_col    = 28    # width of the image

num_image   = len(data)   #10000개
count       = 0     # count for the number of images

#
# normalize the values of the input data to be [0, 1]
#
def normalize(data):

    data_normalized = (data - min(data)) / (max(data) - min(data))

    return(data_normalized)

#
# example of distance function between two vectors x and y
#
def distance(x, y):

    d = (x - y) ** 2
    s = np.sum(d)
    # r = np.sqrt(s)

    return(s)

#
# make a matrix each column of which represents an images in a vector form
#
train_image  = np.empty((size_row * size_col, 6000), dtype=float)
train_label  = np.empty(6000, dtype=int)
test_image  = np.empty((size_row * size_col, 4000), dtype=float)
test_label  = np.empty(4000, dtype=int)

for line in data:
    if count < 6000 :
      line_data   = line.split(',')
      label       = line_data[0]
      im_vector   = np.asfarray(line_data[1:])
      im_vector   = normalize(im_vector)

      train_label[count]       = label       # size : 10000
      train_image[:, count]    = im_vector   # size : 784 * 10000

    else: 
      line_data   = line.split(',')
      label       = line_data[0]
      im_vector   = np.asfarray(line_data[1:])
      im_vector   = normalize(im_vector)

      test_label[count-6000]       = label       # size : 10000
      test_image[:, count-6000]    = im_vector   # size : 784 * 10000

    count += 1
#
# plot first 150 images out of 10,000 with their labels
#
"""
f1 = plt.figure(1)

for i in range(150):

    label       = list_label[i]
    im_vector   = list_image[:, i]
    im_matrix   = im_vector.reshape((size_row, size_col))

    plt.subplot(10, 15, i+1)
    plt.title(label)
    plt.imshow(im_matrix, cmap='Greys', interpolation='None')

    frame   = plt.gca()
    frame.axes.get_xaxis().set_visible(False)
    frame.axes.get_yaxis().set_visible(False)


#plt.show()

#
# plot the average image of all the images for each digit
#
f2 = plt.figure(2)

im_average  = np.zeros((size_row * size_col, 10), dtype=float)
im_count    = np.zeros(10, dtype=int)

for i in range(num_image):

    im_average[:, list_label[i]] += list_image[:, i]
    im_count[list_label[i]] += 1

for i in range(10):

    im_average[:, i] /= im_count[i]

    plt.subplot(2, 5, i+1)
    plt.title(i)
    plt.imshow(im_average[:,i].reshape((size_row, size_col)), cmap='Greys', interpolation='None')

    frame   = plt.gca()
    frame.axes.get_xaxis().set_visible(False)
    frame.axes.get_yaxis().set_visible(False)

plt.show()
"""
train_costs = deque()
test_costs = deque()
theta_0s = deque()
theta_1s = deque()
theta_2s = deque()
bias_0s = deque()
bias_1s = deque()
bias_2s = deque()


print(num_image)  #10000
print(len(train_image))  #784
print(len(train_image[0]))  #6000
print(len(train_label))  #6000
print(len(test_image))  #784
print(len(test_image[0]))  #4000
print(len(test_label))  #4000
#list_label 0~5999까지 training data, 6000~9999까지 test data
#list_image [:,5999]까지 training data, [:,9999]까지 test data

10000
784
6000
6000
784
4000
4000


In [0]:
def forward(theta, value):
  result = np.dot(theta, value)
  return result

In [0]:
def sigmoid(value):
  length = len(value)
  for i in range(0, length):
    value[i] = 1 / (1 + np.exp(-value[i]))
  return value

In [0]:
def addBias(value, bias):
  length = len(bias)
  for i in range(0, length):
    value[i] += bias[i]
  return value

In [0]:
def neuralNet(x, u, v, w, bias0, bias1, bias2):    #data, theta0, theta1, theta2, bias0, bias1, bias2
  y_hat = forward(u, x)
  y_hat = addBias(y_hat, bias0)
  y = sigmoid(y_hat)
  z_hat = forward(v, y)
  z_hat = addBias(z_hat, bias1)
  z = sigmoid(z_hat)
  h_hat = forward(w, z)
  h_hat = addBias(h_hat, bias2)
  h = sigmoid(h_hat)
  return y, z, h

In [0]:
def objectiveFunction(image, label, theta0, theta1, theta2, b0, b1, b2):     #list_image[:, count], list_label -> traing과 test 구분해야겠다.!!
  length = len(label)                    #두는 이유는 trainig 6000, test 4000개  갯수가 달라서
  cost = 0
  for i in range(0, length):
    y, z, outputs = neuralNet(image[:,i], theta0, theta1, theta2, b0, b1, b2)  #size = 10x1 good!!!
    for j in range(0, 10):
      cost += (-label[i] * np.log(outputs[j]) - (1-label[i])*np.log(1-outputs[j])) 
  cost /= length
  return cost

In [0]:
def gradientDescent(image, label, theta0, theta1, theta2, b0, b1, b2):
  lr = 0.1
  length = len(label)
  estimate1 = 0
  for i in range(0, length):
    y, z, outputs = neuralNet(image[:,i], theta0, theta1, theta2, b0, b1, b2)
    for j in range(0, 10):
      b2[j] -= lr * (-label[i] + outputs[j])
      for k in range(0, 49):
        theta2[j][k] -= lr * (-label[i] + outputs[j]) * z[k]
    for j in range(0, 49):
      estimate = 0
      for k in range(0, 10):
        estimate += (-label[i] + outputs[k]) * theta1[k][j]
      b1[j] -= lr * z[j] * (1 - z[j]) * estimate

  


In [52]:
def main():
  mu = 0
  sigma = 1
  theta_0 = np.random.normal(mu, sigma, size=(196,784))   #size = 196x784
  theta_1 = np.random.normal(mu, sigma, size=(49,196))    #size = 49x196
  theta_2 = np.random.normal(mu, sigma, size=(10,49))     #size = 10x49
  
  b_0 = np.random.normal(mu, sigma, 196)                  #size = 196x1
  b_1 = np.random.normal(mu, sigma, 49)                   #size = 49x1
  b_2 = np.random.normal(mu, sigma, 10)                   #size = 10x1

  #training step
  for iter in range(0, 2):
    train_cost = objectiveFunction(test_image, test_label, theta_0, theta_1, theta_2, b_0, b_1, b_2)  #print(cost)
    train_costs.append(train_cost)
    gradientDescent(test_image, test_label, theta_0, theta_1, theta_2, b_0, b_1, b_2)
    print(b_1)

if __name__ == "__main__":
  main()

[ 16.23977142 -26.6120368    5.52205977 -23.79286087 -19.00820672
 -25.07650356  22.02755007  14.08901298  13.11098816  -8.43015341
  10.84719461 -30.93914615  14.58745115  24.15171183 -27.82608124
  17.29808801  17.51469967   7.26738193  -8.86423874  12.06814132
  21.29268026   9.191606    18.50510166  16.48217604 -15.90655281
 -17.22657985  12.68246247   1.84838733  15.28496426  17.27586481
  23.44320408 -14.21392708  -9.25632388 -21.73689503 -29.44620323
  18.72459804  14.58581408  11.15024373  21.408091   -17.49418439
  16.01610535 -16.73173349  20.73323943  26.19822444  -6.23073764
   5.76330298  20.66624505  11.54700753   0.94818238]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in log
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in double_scalars
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in multiply
  import sys


[ 17.43722634 -27.96859028   6.80008816 -25.19373959 -19.85287185
 -25.87473847  22.90908485  15.08773985  16.47927682 -10.10863615
  12.4227129  -32.02681074  16.23625689  25.46852083 -28.98950403
  18.71653873  19.89885988   8.82052014 -10.16440809  13.02744943
  22.6753508   10.41980852  20.17023389  17.67184016 -18.10009678
 -18.62688382  13.77812779   2.80978783  22.37671407  18.48454461
  24.51580913 -15.29373362 -10.15513604 -23.09767639 -31.08126276
  19.69707014  16.47722412  12.27173358  23.15275383 -18.65964969
  17.69854535 -17.90564193  22.17751225  28.10232281  -7.77733154
   8.65358075  21.5697856   12.68143905   2.02377643]


In [0]:
a = np.random.normal(0,1,size=(2,3))
b = np.random.normal(0,1,(2))
print(a)
print(b)
b = np.tile(b.T,3)
#print(b)
b= b.reshape(2,-1)
print(b)
c = np.add(a,b)
print(c)